# DEMO: Getting Started with Regression

### 1. Import Packages and Connect to the CAS Server

Visit the documentation for the SWAT [(SAS Scripting Wrapper for Analytics Transfer)](https://sassoftware.github.io/python-swat/index.html) package.

In [1]:
## Import packages
import swat
import pandas as pd

## Set options
pd.set_option('max_columns', None)
#pd.set_option('display.float_format', lambda x: '%.6f' % x)

## Connect to CAS
conn = swat.CAS('server.demo.sas.com', 30571, 'student', 'Metadata0', name='py04d03')

## Function to load the loans_raw table into memory if necessary
def loadloans():
    conn.loadTable(path='loans_raw.sashdat', caslib='casuser',
                   casOut={'name':'loans_raw',
                           'caslib':'casuser',
                           'promote':True})

### 2. Explore Available CAS Tables and Data Source Files

a. Use the **tableInfo** action to view all available in-memory tables in the casuser caslib. If the **loans_raw** CAS table is not available, uncomment the statement and execute the loadloans function.

In [2]:
#loadloans()
conn.tableInfo(caslib='casuser')

,Name,Label,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,CARS,2004 Car Data,428,15,0,utf-8,2021-11-29T14:03:24+00:00,2021-11-29T14:03:24+00:00,2021-12-01T18:06:10+00:00,UTF8,1.953814e+09,1.953814e+09,1.954001e+09,1,0,0,0,cars.sas7bdat,CASUSER(student),0,student,,2021-11-19T20:35:37+00:00,1.952973e+09
1,LOANS_RAW,,18124151,20,0,utf-8,2021-11-30T21:00:16+00:00,2021-11-30T21:00:16+00:00,2021-12-01T21:14:07+00:00,UTF8,1.953925e+09,1.953925e+09,1.954012e+09,1,0,0,0,loans_raw.sashdat,CASUSER(student),0,student,,2021-11-19T20:31:23+00:00,1.952973e+09


b. Reference the **loans_raw** CAS table.

In [3]:
mTbl = conn.CASTable('loans_raw', caslib='casuser', where = 'Category = "Mortgage"')

c. Preview the **loans_raw** table of rows where the **Category** is *Mortgage* using the head method.

In [4]:
mTbl.head().loc[:,['Amount','LoanStatus','LoanGrade','Salary', 'InterestRate']]

,Amount,LoanStatus,LoanGrade,Salary,InterestRate
0,0.0,Cancelled,B,298003.0,4.56
1,0.0,Current,A,188995.0,3.49
2,278658.6,Current,B,171926.0,4.80
3,0.0,Default,D,40221.0,5.21
4,0.0,Current,A,191457.0,3.80


**Can we predict the mortgage Amount based on the above columns?**

### 3. Regression

a. Load and preview the [regression](https://go.documentation.sas.com/doc/en/pgmsascdc/v_018/casactstat/casactstat_regression_toc.htm) action set.

In [5]:
conn.loadActionSet('regression')
conn.regression?

NOTE: Added action set 'regression'.


Signature:   conn.regression(*args, **kwargs)
Type:        Regression
String form: <swat.cas.actions.Regression object at 0x0000013001CFA100>
File:        c:\programdata\anaconda3\lib\site-packages\swat\cas\actions.py
Docstring:  
Regression

Actions
-------
regression.genmod              : Fits generalized linear regression models
regression.genmodscore         : creates a table on the server that contains
                                 results from scoring observations by using a
                                 fitted model
regression.glm                 : Fits linear regression models using the method
                                 of least squares
regression.glmscore            : creates a table on the server that contains
                                 results from scoring observations by using a
                                 fitted model
regression.logistic            : Fits logistic regression models
regression.logisticassociation : computes indices of rank correlation

b.

In [ ]:
classCols = ['Type','Origin']


mTbl.glm(
    target = 'MSRP',
    inputs = ['Salary', 'InterestRate'] + classCols,
    nominals = classCols,
    selection = {'method':'BACKWARD'}
)

For more information on Machine Learning with Python, check out the [SAS® Viya® and Python Integration for Machine Learning](https://support.sas.com/edu/schedules.html?crs=POSI34&ctry=US) course.

### 4. Terminate the CAS Session

a. It's best practice to always terminate the CAS session when you are done.

In [7]:
conn.terminate()